In [ ]:
# 1. PCA Programming

## 1.1. Download Dataset

In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from PCA import MyPCA
from ClusteringAlgorithm.KMeans import MyKMeans
from ClusteringAlgorithm.GMM import MyGMM
from Utils.Utils import *

warnings.filterwarnings('ignore')

iris = load_iris()
X = pd.DataFrame(iris['data'], columns = iris['feature_names'])
y = pd.DataFrame(iris['target'], columns = ['species'])

## 1.2. EDA

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
X.describe()

In [ ]:
X_all = pd.concat([X, y], axis = 1)

sns.pairplot(X_all, hue='species', palette='tab10', diag_kind='hist')

# 2. PCA

In [ ]:
X, y = np.array(X), np.array(y)

In [ ]:
my_pca = MyPCA(n_components = 2)
my_pca.fit(X)

In [ ]:
# Transform
X_transformed = my_pca.transform(X)
print("\nDữ liệu sau khi biến đổi (X_transformed) có shape:", X_transformed.shape)
print("Một vài mẫu dữ liệu sau khi biến đổi:")
print(X_transformed[:5, :])

# Check PCA của sklearn

In [ ]:
from sklearn.decomposition import PCA as SklearnPCA

print("--- Kiểm tra với sklearn.decomposition.PCA ---")
# 1. Khởi tạo PCA của sklearn
sklearn_pca = SklearnPCA(n_components=2)

# 2. Khớp mô hình với dữ liệu X
sklearn_pca.fit(X)

# 3. Biến đổi dữ liệu X
X_transformed_sklearn = sklearn_pca.transform(X)

print("\nTỷ lệ phương sai giải thích (EVR) cho từng thành phần (sklearn.explained_variance_ratio_):")
# Đây là EVR cho n_components đã chọn
print(sklearn_pca.explained_variance_ratio_)
# Tương đương với self.explained_variance_ratio_[:n_components] của MyPCA

print("\nPhương sai giải thích bởi từng thành phần (sklearn.explained_variance_):")
# Đây là các trị riêng tương ứng với n_components đã chọn, không phải tỷ lệ
print(sklearn_pca.explained_variance_)
# Tương đương với self.explained_variance_[:n_components] của MyPCA

# sklearn không có thuộc tính CEVR trực tiếp, nhưng có thể dễ dàng tính từ EVR
cumulative_explained_variance_ratio_sklearn = np.cumsum(sklearn_pca.explained_variance_ratio_)
print("\nTỷ lệ phương sai giải thích tích lũy (CEVR) tính từ sklearn.explained_variance_ratio_:")
for i, cum_ratio in enumerate(cumulative_explained_variance_ratio_sklearn):
    print(f"  PC1 đến PC{i+1}: {cum_ratio:.4f}")
print(f"Tổng phương sai được giải thích bởi {sklearn_pca.n_components_} thành phần: {cumulative_explained_variance_ratio_sklearn[-1]:.4f}")
# Giá trị cuối cùng này tương đương với self.cumulative_explained_variance_ratio_[n_components-1] của MyPCA

print("\nDữ liệu sau khi biến đổi (X_transformed_sklearn) có shape:", X_transformed_sklearn.shape)
print("Một vài mẫu dữ liệu sau khi biến đổi (sklearn):")
print(X_transformed_sklearn[:5, :]) # Tương đương với X_transformed của MyPCA
print("---------------------------------------------")

# 3. K-Means Clustering

## 3.1. Data Preprocessing

In [ ]:
X = pd.read_csv('../Dataset/ABIDE2.csv', index_col = 0)
y = X['group']

In [ ]:
X.head(5)

In [ ]:
X.info()

In [ ]:
X.describe()

In [ ]:
X['site'].describe()

### **Nhận xét**

* Ta sẽ cần chuyển các giá trị ở cột **"site"** từ Categorical Features thành Numerical Features. Vì ở cột này là Nominal Feature và có 17 giá trị nên one-hot encoding là phù hợp nhất.

* Cột **"group"** sẽ được chuyển từ 'Cancer'/'Normal' thành dạng số (ví dụ: 1/0).

* **Standard Scaler**: Phải được áp dụng vì các features có mean và variance rất khác nhau.

In [ ]:
X_num = X.drop(columns = ['site', 'group'])

# Encode cột side
site_encoded = pd.get_dummies(X['site'], prefix='site', dtype=int)

# Encode cột group
group_mapping = {'Normal': 0, 'Cancer': 1}
y = X['group'].map(group_mapping).values

# Kết hợp features
X = pd.concat([X_num, site_encoded], axis=1)

# Áp dụng Standard Scaler
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X))

In [ ]:
X.head()

In [ ]:
transformed_data = transform_features(
    data_path="../Dataset/ABIDE2.csv",
    skew_threshold=0.5,
    poly_degree=2,
    poly_threshold=0.1,
    visualize=False,
    interaction_terms=True
)

In [ ]:
transformed_data.head()

In [ ]:
def evaluate_pca_kmeans(X, y, n_components_list=[1, 2, 5, 10, 20, 50, 100, 150], n_clusters=2, random_state=42):
    """
    Đánh giá hiệu quả của PCA với các số components khác nhau kết hợp KMeans

    Args:
        X: dữ liệu gốc
        y: nhãn thật
        n_components_list: list số components muốn thử
        n_clusters: số cụm cho KMeans
        random_state: seed cho việc tái tạo kết quả

    Returns:
        DataFrame chứa các metric đánh giá
    """
    from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, silhouette_score

    results = []

    for n_comp in n_components_list:
        # Kiểm tra nếu số components lớn hơn số features
        if n_comp > X.shape[1]:
            print(f"Bỏ qua {n_comp} components vì lớn hơn số features ({X.shape[1]})")
            continue

        print(f"Đang đánh giá với {n_comp} components...")

        # Thực hiện PCA
        pca = MyPCA(n_components=n_comp)
        X_pca = pca.fit_transform(X)

        # Thực hiện KMeans
        kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
        pred_labels = kmeans.fit_predict(X_pca)

        # Đảm bảo nhãn dự đoán khớp với nhãn thật
        # (vì KMeans có thể gán nhãn 0/1 ngược với thực tế)
        if accuracy_score(y, pred_labels) < 0.5:
            pred_labels = 1 - pred_labels

        # Tính các metrics
        metrics = {
            'n_components': n_comp,
            'accuracy': accuracy_score(y, pred_labels),
            'recall': recall_score(y, pred_labels),
            'precision': precision_score(y, pred_labels),
            'f1': f1_score(y, pred_labels),
            'silhouette': silhouette_score(X_pca, pred_labels)
        }

        results.append(metrics)

    # Tạo DataFrame kết quả
    results_df = pd.DataFrame(results)

    # Visualize kết quả
    plt.figure(figsize=(15, 6))

    # Line plot cho các metrics
    plt.subplot(1, 2, 1)
    for metric in ['accuracy', 'recall', 'precision', 'f1']:
        plt.plot(results_df['n_components'], results_df[metric],
                marker='o', label=metric.capitalize())
    plt.xscale('log')
    plt.xlabel('Số components (log scale)')
    plt.ylabel('Score')
    plt.title('Các metrics theo số components')
    plt.legend()
    plt.grid(True)

    # Silhouette score
    plt.subplot(1, 2, 2)
    plt.plot(results_df['n_components'], results_df['silhouette'],
            marker='o', color='purple', label='Silhouette')
    plt.xscale('log')
    plt.xlabel('Số components (log scale)')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Score theo số components')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    return results_df

# Sử dụng hàm
results = evaluate_pca_kmeans(X, y)
print("\nKết quả chi tiết:")
print(results.to_string(index=False))